In [ ]:
# B3_Configurable_Minute_SOL_Optuna_V6_Precision.py # <-- Renamed
# Optuna maximizes Precision@Threshold in CV folds, uses SMOTE & updated features.

import pandas as pd
import numpy as np
import time
import os
import warnings
import traceback
from datetime import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
import optuna
import pandas_ta as ta
from imblearn.over_sampling import SMOTE

# Modeling Imports
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.exceptions import UndefinedMetricWarning

# --- Suppress Warnings ---
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ==============================================================================
# --- Configuration ---
# ==============================================================================
CSV_FILE_PATH = 'SOL_minagg.csv'; SYMBOL_NAME = 'SOL'
PREDICTION_WINDOW_MINUTES = int(1 * 60); TARGET_THRESHOLD_PCT = 0.24 # Predict +2% in 30m
TRAIN_WINDOW_MINUTES = 8 * 60; STEP_MINUTES = 4 * 60; TEST_WINDOW_FRACTION = 0.2
XGB_FIXED_PARAMS = {"objective":"binary:logistic", "eval_metric":"logloss", "use_label_encoder": False, "random_state":42, "tree_method":"gpu_hist", "predictor":"gpu_predictor", "gpu_id":0, "n_jobs":-1, "n_estimators":150}
N_OPTUNA_TRIALS = 50; OPTUNA_CV_SPLITS = 3
# Optuna will optimize PRECISION calculated at this threshold
OPTUNA_EVAL_THRESHOLD = 0.25 # Keep lower threshold for rare positive class eval
SMOTE_K_NEIGHBORS = 5
PROBABILITY_THRESHOLD_RANGE = (0.05, 0.95); PROBABILITY_THRESHOLD_STEP = 0.05

# ==============================================================================
# --- Derived Variables ---
# ==============================================================================
TEST_WINDOW_MINUTES = max(1, int(TEST_WINDOW_FRACTION * TRAIN_WINDOW_MINUTES))
THRESHOLD_SEARCH_RANGE = np.arange(PROBABILITY_THRESHOLD_RANGE[0], PROBABILITY_THRESHOLD_RANGE[1], PROBABILITY_THRESHOLD_STEP)
epsilon = 1e-9

# ==============================================================================
# --- Feature Engineering Function (calculate_features_min_rare_event - Unchanged) ---
# ==============================================================================
def calculate_features_min_rare_event(df_input):
    df = df_input.copy()
    # print(f"  Feature Eng Start: Initial rows = {len(df)}") # Optional debug
    essential_cols = ['open', 'high', 'low', 'close', 'volumefrom']
    initial_nan_check = df[essential_cols].isnull().sum()
    if initial_nan_check.sum() > 0: df = df.dropna(subset=essential_cols)
    if df.empty: return df
    base_cols_numeric = ['open', 'high', 'low', 'close', 'volumefrom', 'volumeto']
    for col in base_cols_numeric:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
        else: df[col] = 0
    df = df.dropna(subset=essential_cols)
    if df.empty: return df
    df['price_change_1m_temp'] = df['close'].pct_change(periods=1)
    df['body_abs'] = abs(df['close'] - df['open'])
    df['range'] = df['high'] - df['low']
    df['body_ratio'] = (df['body_abs'] / (df['range'] + epsilon)).clip(0, 1)
    with np.errstate(divide='ignore', invalid='ignore'):
        df['price_range_pct'] = (df['range'] / df['close'].replace(0, np.nan)) * 100
        df['oc_change_pct'] = (df['close'] - df['open']) / df['open'].replace(0, np.nan) * 100
    min_periods_rolling = 2
    for p in [10, 30, 60]: df[f'ma_{p}m'] = df['close'].rolling(p, min_periods=min_periods_rolling).mean()
    for p in [30, 60]: df[f'rolling_std_{p}m'] = df['price_change_1m_temp'].rolling(p, min_periods=p//2).std() * 100
    lag_periods_price_min = [1, 3, 5, 10, 15, 30, 60]; lag_periods_volume_min = [1, 3, 5, 10, 15, 30, 60]
    for lag in lag_periods_price_min: df[f'lag_{lag}m_price_return'] = df['price_change_1m_temp'].shift(lag) * 100
    df['volume_return_1m'] = df['volumefrom'].pct_change(periods=1).replace([np.inf, -np.inf], 0) * 100
    for lag in lag_periods_volume_min: df[f'lag_{lag}m_volume_return'] = df['volume_return_1m'].shift(lag)
    vol_ma_period = 20; df[f'vol_ma_{vol_ma_period}m'] = df['volumefrom'].rolling(vol_ma_period, min_periods=vol_ma_period//2).mean()
    df['vol_spike_ratio'] = df['volumefrom'] / (df[f'vol_ma_{vol_ma_period}m'] + epsilon)
    body_ma_period = 20; df[f'body_ma_{body_ma_period}m'] = df['body_abs'].rolling(body_ma_period, min_periods=body_ma_period//2).mean()
    df['body_spike_ratio'] = df['body_abs'] / (df[f'body_ma_{body_ma_period}m'] + epsilon)
    df['prev_close']=df['close'].shift(1); df['hml']=df['high']-df['low']; df['hmpc']=np.abs(df['high']-df['prev_close']); df['lmpc']=np.abs(df['low']-df['prev_close'])
    df['tr']=df[['hml','hmpc','lmpc']].max(axis=1); atr_periods_min = [14]; min_p_atr = 10;
    for p in atr_periods_min: df[f'atr_{p}m'] = df['tr'].rolling(p, min_periods=min_p_atr).mean()
    df = df.drop(columns=['prev_close', 'hml', 'hmpc', 'lmpc', 'tr', 'body_abs', 'range', f'vol_ma_{vol_ma_period}m', f'body_ma_{body_ma_period}m'], errors='ignore')
    df['atr_norm'] = df[f'atr_{atr_periods_min[0]}m'] / (df['close'] + epsilon) if f'atr_{atr_periods_min[0]}m' in df else np.nan
    # print("  Calculating TA features (RSI, Stoch, MACD, BBands)...") # Optional debug
    try:
        min_ta_warmup = 30
        if len(df) >= min_ta_warmup:
            df.ta.rsi(length=14, append=True)
            if 'RSI_14' in df.columns: df['rsi_14_oversold']=(df['RSI_14']<30).astype(int); df['rsi_14_overbought']=(df['RSI_14']>70).astype(int); df['rsi_ob_confirm']=((df['RSI_14']>70)&(df['close']>df['open'])).astype(int); df['rsi_os_confirm']=((df['RSI_14']<30)&(df['close']<df['open'])).astype(int)
            df.ta.stoch(k=14, d=3, smooth_k=3, append=True)
            if 'STOCHk_14_3_3' in df.columns: df['stoch_k_oversold']=(df['STOCHk_14_3_3']<20).astype(int); df['stoch_k_overbought']=(df['STOCHk_14_3_3']>80).astype(int)
            df.ta.macd(fast=12, slow=26, signal=9, append=True)
            if 'MACDh_12_26_9' in df.columns: df['macd_hist_positive']=(df['MACDh_12_26_9']>0).astype(int); df['macd_hist_increasing']=(df['MACDh_12_26_9']>df['MACDh_12_26_9'].shift(1)).astype(int)
            df.ta.bbands(length=20, std=2, append=True)
            if 'BBP_20_2.0' in df.columns: df['bbp_near_upper']=(df['BBP_20_2.0']>0.9).astype(int); df['bbp_near_lower']=(df['BBP_20_2.0']<0.1).astype(int)
            if 'RSI_14' in df.columns:
                for n_div in [30]:
                    if len(df) > n_div: min_price_n=df['low'].rolling(n_div,min_periods=n_div//2).min(); min_rsi_n=df['RSI_14'].rolling(n_div,min_periods=n_div//2).min(); price_lower_low=df['low']<min_price_n.shift(1); rsi_higher_low=df['RSI_14']>min_rsi_n.shift(1); df[f'rsi_bull_div_{n_div}m']=(price_lower_low&rsi_higher_low).astype(int)
                    else: df[f'rsi_bull_div_{n_div}m']=0
        # else: print(f"  Warning: Insufficient data ({len(df)} rows) for TA warmup.") # Optional debug
    except Exception as e_ta: print(f"!! Error calculating TA features: {e_ta}")
    cols_to_drop_intermediate = ['price_change_1m_temp', 'volume_return_1m']
    df = df.drop(columns=[col for col in cols_to_drop_intermediate if col in df.columns], errors='ignore')
    numeric_cols = df.select_dtypes(include=np.number).columns
    if df[numeric_cols].isin([np.inf, -np.inf]).any().any(): df = df.replace([np.inf, -np.inf], np.nan)
    # print(f"  Feature Eng End: Total columns = {df.shape[1]}, Rows = {len(df)}") # Optional debug
    return df

# ==============================================================================
# --- Optuna Objective Function (Optimizing for PRECISION) ---
# ==============================================================================
def objective(trial, X, y, fixed_params, cv_strategy, k_neighbors):
    """Objective function for Optuna using SMOTE within CV folds, maximizing Precision@Threshold.""" # MODIFIED DOCSTRING

    # Define hyperparameter search space (scale_pos_weight is removed)
    param = {
        "max_depth":        trial.suggest_int("max_depth", 5, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "reg_alpha":        trial.suggest_float("reg_alpha", 1e-3, 0.5, log=True),
        "reg_lambda":       trial.suggest_float("reg_lambda", 1.0, 10.0, log=True),
        "gamma":            trial.suggest_float("gamma", 0, 0.5),
        "subsample":        trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "learning_rate":    trial.suggest_float("learning_rate", 0.03, 0.15, log=True),
        # REMOVED 'scale_pos_weight'
    }
    xgb_params = {key: val for key, val in fixed_params.items() if key != 'scale_pos_weight'}
    xgb_params.update(param)

    cv_scores = [] # Store precision scores for this trial
    try:
        y_np = y.to_numpy() if isinstance(y, pd.Series) else np.array(y)
        X_np = X.to_numpy() if isinstance(X, pd.DataFrame) else np.array(X)

        for fold, (train_idx, val_idx) in enumerate(cv_strategy.split(X_np, y_np)):
            X_train_fold, X_val_fold = X_np[train_idx], X_np[val_idx]
            y_train_fold, y_val_fold = y_np[train_idx], y_np[val_idx]

            # --- SMOTE ---
            minority_class_count = np.sum(y_train_fold == 1)
            X_train_resampled, y_train_resampled = X_train_fold, y_train_fold
            if minority_class_count >= k_neighbors + 1:
                try:
                    smote = SMOTE(random_state=fixed_params.get("random_state", 42) + fold, k_neighbors=k_neighbors)
                    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_fold, y_train_fold)
                except Exception as e_smote:
                    print(f"  SMOTE Error fold {fold+1}: {e_smote}")
            # --- End SMOTE ---

            if len(np.unique(y_train_resampled)) < 2: cv_scores.append(0.0); continue

            model = xgb.XGBClassifier(**xgb_params)
            model.fit(X_train_resampled, y_train_resampled, verbose=False)

            # --- MODIFIED EVALUATION: Use PRECISION ---
            preds_proba = model.predict_proba(X_val_fold)[:, 1]
            preds_binary = (preds_proba >= OPTUNA_EVAL_THRESHOLD).astype(int)
            # Calculate Precision instead of F1
            precision = precision_score(y_val_fold, preds_binary, zero_division=0)
            cv_scores.append(precision)
            # --- END MODIFICATION ---

            # Optuna Pruning Integration
            trial.report(precision, fold) # Report precision for pruning
            if trial.should_prune(): raise optuna.exceptions.TrialPruned()

        # Calculate the average PRECISION score across valid folds
        average_score = np.mean(cv_scores) if cv_scores else 0.0 # Store average precision

    except optuna.exceptions.TrialPruned: raise
    except Exception as e: print(f"Error in Optuna trial {trial.number}, fold {fold}: {e}"); return 0.0
    return average_score if not np.isnan(average_score) else 0.0

# ==============================================================================
# --- Main Script Logic ---
# ==============================================================================

overall_start_time = time.time()

# --- 1. Load Data ---
print("--- Data Loading ---"); print(f"Loading data from: {CSV_FILE_PATH}...")
try:
    df_data = pd.read_csv(CSV_FILE_PATH, parse_dates=['timestamp'])
    df_data = df_data.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
    print(f"Loaded {len(df_data)} rows for {SYMBOL_NAME}.")
except Exception as e: print(f"Error loading data: {e}"); exit()
min_data_needed = TRAIN_WINDOW_MINUTES + TEST_WINDOW_MINUTES + STEP_MINUTES
if len(df_data) < min_data_needed: print(f"Error: Insufficient initial data ({len(df_data)} < {min_data_needed} needed)."); exit()


# --- 2. Feature Engineering ---
print(f"\n--- Feature Engineering for {SYMBOL_NAME} ---")
start_fe = time.time()
df_data = calculate_features_min_rare_event(df_data) # Use the function with new features
if df_data.empty: print(f"Error: Feature calculation resulted in empty DataFrame."); exit()
print(f"Feature engineering complete. Took {time.time() - start_fe:.2f}s.")

# --- 3. Define Target Variable ---
print("\n--- Target Definition ---")
print(f"Defining target: {PREDICTION_WINDOW_MINUTES}m return >= {TARGET_THRESHOLD_PCT}%...")
target_col = f'target_return_{PREDICTION_WINDOW_MINUTES}m'
df_data[target_col] = df_data['close'].shift(-PREDICTION_WINDOW_MINUTES).sub(df_data['close']).div(df_data['close'].replace(0, np.nan)).mul(100)
target_occurrence = df_data[target_col].notna() & (df_data[target_col] >= TARGET_THRESHOLD_PCT)
print(f"  Raw positive target occurrence (before NaN drop): {target_occurrence.mean()*100:.2f}%")

# --- 4. Prepare Data for Modeling ---
print("\n--- Data Preparation ---")
base_cols_ohlcv = ['open', 'high', 'low', 'close', 'volumefrom', 'volumeto']
cols_to_keep_final = ['timestamp', target_col]
potential_feature_cols = [col for col in df_data.columns if col not in cols_to_keep_final and col not in base_cols_ohlcv and not col.startswith('STOCHd') and not col.startswith('MACDs')]
numeric_feature_cols = df_data[potential_feature_cols].select_dtypes(include=np.number).columns.tolist()
final_feature_cols = numeric_feature_cols
if not final_feature_cols: print("Error: No numeric features found after selection."); exit()
cols_to_select = final_feature_cols + [col for col in cols_to_keep_final if col in df_data.columns]
df_model_ready = df_data[cols_to_select].copy()

print("Applying final NaN/Inf Handling...")
initial_rows = len(df_model_ready)
df_model_ready = df_model_ready.dropna(subset=final_feature_cols + [target_col])
final_rows = len(df_model_ready)
print(f"NaN Handling: Dropped {initial_rows - final_rows} rows with NaNs.")
if df_model_ready[final_feature_cols].isin([np.inf, -np.inf]).any().any():
    inf_count = df_model_ready[final_feature_cols].isin([np.inf, -np.inf]).sum().sum()
    print(f"Replacing {inf_count} final infinites...")
    df_model_ready.replace([np.inf, -np.inf], np.nan, inplace=True)
    rows_b4 = len(df_model_ready); df_model_ready = df_model_ready.dropna(subset=final_feature_cols)
    print(f"Dropped {rows_b4 - len(df_model_ready)} more rows after Inf handling.")
if df_model_ready.empty: print(f"Error: DataFrame empty after final NaN/Inf handling."); exit()

X = df_model_ready[final_feature_cols]
y_binary = (df_model_ready[target_col] >= TARGET_THRESHOLD_PCT).astype(int)
timestamps = df_model_ready['timestamp']
print(f"\nFinal feature matrix shape: {X.shape}, Target shape: {y_binary.shape}")
print(f"Using {len(final_feature_cols)} features.")
print(f"Positive Target Rate in Final Data: {y_binary.mean()*100:.2f}%") # Check final balance

# --- 5. SLIDING Window Backtesting ---
print(f"\n--- Starting SLIDING Window Backtest for {SYMBOL_NAME} ---")
print(f"!!! Using Optuna (TimeSeriesSplit CV + SMOTE, optimizing Precision) + Rare Event Features !!!") # <-- Updated print

if len(X) < TRAIN_WINDOW_MINUTES + STEP_MINUTES:
     print(f"Error: Not enough data after pre-processing ({len(X)} rows) for train window ({TRAIN_WINDOW_MINUTES}) + step ({STEP_MINUTES})."); exit()

all_predictions_proba = []; all_actual = []; backtest_timestamps = []
all_best_params = []
num_steps = 0
start_index_loop = TRAIN_WINDOW_MINUTES
end_index_loop = len(X) - TEST_WINDOW_MINUTES + 1

print(f"Train Window: {TRAIN_WINDOW_MINUTES}m, Step: {STEP_MINUTES}m, Test Window: {TEST_WINDOW_MINUTES}m, Optuna Trials: {N_OPTUNA_TRIALS}, CV Splits: {OPTUNA_CV_SPLITS}")
loop_start_time = time.time()

for i in range(start_index_loop, end_index_loop, STEP_MINUTES):
    step_start_time = time.time()
    train_idx_start = i - TRAIN_WINDOW_MINUTES
    train_idx_end = i
    test_idx_start = i
    test_idx_end = min(i + TEST_WINDOW_MINUTES, len(X))

    if test_idx_start >= test_idx_end: print(f"Stopping loop: Test window invalid."); break

    X_train_roll = X.iloc[train_idx_start : train_idx_end]
    y_train_roll = y_binary.iloc[train_idx_start : train_idx_end]
    X_test_roll = X.iloc[test_idx_start : test_idx_end]
    y_test_roll_actual_series = y_binary.iloc[test_idx_start : test_idx_end]
    step_timestamps = timestamps.iloc[test_idx_start : test_idx_end]

    if y_test_roll_actual_series.empty: print(f"Warning: Step {i}, empty test actuals."); continue
    current_timestamp = step_timestamps.iloc[0]
    if X_train_roll.empty or len(np.unique(y_train_roll)) < 2: print(f"Warning: Step {i}, invalid training data."); continue

    print(f"\n--- Step {num_steps + 1} ({current_timestamp}) ---")
    print(f"  Train: [{train_idx_start}:{train_idx_end-1}]; Test: [{test_idx_start}:{test_idx_end-1}]")
    print(f"  Training data balance before SMOTE: {y_train_roll.mean()*100:.2f}% positive")

    # --- Hyperparameter Tuning with Optuna ---
    print(f"  Running Optuna ({N_OPTUNA_TRIALS} trials, cv={OPTUNA_CV_SPLITS} TimeSeriesSplit+SMOTE, scoring Precision@{OPTUNA_EVAL_THRESHOLD})...") # <-- Updated print
    optuna_start_time = time.time()
    try:
        cv_strategy = TimeSeriesSplit(n_splits=OPTUNA_CV_SPLITS)
        pruner = optuna.pruners.MedianPruner(n_warmup_steps=5, n_min_trials=10)
        sampler = optuna.samplers.TPESampler(seed=i)
        study = optuna.create_study(direction='maximize', pruner=pruner, sampler=sampler) # Maximize Precision
        obj_func = lambda trial: objective(trial, X_train_roll, y_train_roll, XGB_FIXED_PARAMS, cv_strategy, SMOTE_K_NEIGHBORS)

        study.optimize(obj_func, n_trials=N_OPTUNA_TRIALS, n_jobs=1, show_progress_bar=False)

        best_params_step = study.best_params
        best_score_step = study.best_value # Best average Precision from CV

        print(f"  Optuna finished in {time.time() - optuna_start_time:.2f}s.")
        print(f"  Best Params: {best_params_step}, Best CV Precision(@{OPTUNA_EVAL_THRESHOLD}): {best_score_step:.4f}") # <-- Updated print
        all_best_params.append({'step': num_steps + 1, 'params': best_params_step, 'cv_precision': best_score_step, 'timestamp': current_timestamp}) # <-- Updated key

        # --- Fit final model for the step ---
        final_model_params = {**XGB_FIXED_PARAMS, **best_params_step}
        print("  Fitting final model on original training data for step...")
        model_roll = xgb.XGBClassifier(**final_model_params)
        model_roll.fit(X_train_roll, y_train_roll, verbose=False)

        # --- Predict probabilities for the test window ---
        prob_roll_window = model_roll.predict_proba(X_test_roll)[:, 1]

        # --- Store results ---
        all_predictions_proba.extend(prob_roll_window)
        all_actual.extend(y_test_roll_actual_series.tolist())
        backtest_timestamps.extend(step_timestamps.tolist())
        num_steps += 1

    except ValueError as ve: print(f"!! Value Error at step {i}: {ve}"); continue
    except Exception as e_step: print(f"!! Error at step {i}: {e_step}"); traceback.print_exc(); continue

    step_end_time = time.time()
    print(f"  Step {num_steps} finished in {step_end_time - step_start_time:.2f}s total.")

loop_end_time = time.time()
print(f"\nBacktesting loop finished. Completed {num_steps} steps in {(loop_end_time - loop_start_time)/60:.2f} minutes.")

# --- 6. Evaluate Backtesting Results with PTT ---
# NOTE: Final PTT evaluation still optimizes for F1 score by default.
# You could change the logic here to optimize for Precision or another metric if desired.
if num_steps > 0 and len(all_predictions_proba) == len(all_actual) and len(all_predictions_proba) == len(backtest_timestamps):
    print(f"\n--- Evaluating Results for {SYMBOL_NAME} with Probability Threshold Tuning (Optimizing F1 overall) ---") # Clarified PTT goal
    print(f"Threshold search range: {THRESHOLD_SEARCH_RANGE}")
    best_threshold_f1 = 0.5; best_f1_overall = -1.0 # Vars specific to F1 PTT
    results_per_threshold = {}
    probabilities_np = np.array(all_predictions_proba)
    actual_np = np.array(all_actual)

    for t in THRESHOLD_SEARCH_RANGE:
        predictions_thresh = (probabilities_np >= t).astype(int)
        # Calculate all metrics for reporting
        if np.sum(actual_np) == 0 and np.sum(predictions_thresh) == 0: acc_t, pre_t, rec_t, f1_t = 1.0, 1.0, 1.0, 1.0
        elif np.sum(actual_np) > 0 and np.sum(predictions_thresh) == 0: acc_t = accuracy_score(actual_np, predictions_thresh); pre_t, rec_t, f1_t = 0.0, 0.0, 0.0
        elif np.sum(actual_np) == 0 and np.sum(predictions_thresh) > 0: acc_t = accuracy_score(actual_np, predictions_thresh); pre_t, rec_t, f1_t = 0.0, 0.0, 0.0
        else:
             acc_t = accuracy_score(actual_np, predictions_thresh); pre_t = precision_score(actual_np, predictions_thresh, zero_division=0)
             rec_t = recall_score(actual_np, predictions_thresh, zero_division=0); f1_t = f1_score(actual_np, predictions_thresh, zero_division=0)
        results_per_threshold[round(t, 2)] = {'f1': f1_t, 'acc': acc_t, 'pre': pre_t, 'rec': rec_t}

        # Find best threshold based on F1 score
        if f1_t >= best_f1_overall:
             if f1_t > best_f1_overall or abs(t - 0.5) < abs(best_threshold_f1 - 0.5): # Tie-break for F1
                  best_f1_overall = f1_t; best_threshold_f1 = t

    print(f"\nBest Threshold for {SYMBOL_NAME} (Maximizing F1): {best_threshold_f1:.2f} (Yielding F1 Score: {best_f1_overall:.4f})")

    # Calculate final metrics using the threshold that optimized F1
    final_predictions_optimized = (probabilities_np >= best_threshold_f1).astype(int)
    final_accuracy = accuracy_score(actual_np, final_predictions_optimized)
    final_precision = precision_score(actual_np, final_predictions_optimized, zero_division=0)
    final_recall = recall_score(actual_np, final_predictions_optimized, zero_division=0)
    final_f1 = f1_score(actual_np, final_predictions_optimized, zero_division=0) # This is best_f1_overall

    print(f"\n--- Final Performance Metrics for {SYMBOL_NAME} (Threshold Optimized for F1) ---")
    print(f"Target: {PREDICTION_WINDOW_MINUTES}m return >= {TARGET_THRESHOLD_PCT}%")
    print(f"Windowing: Train={TRAIN_WINDOW_MINUTES}m, Step={STEP_MINUTES}m, Test Window={TEST_WINDOW_MINUTES}m")
    print(f"Hyperparameter Tuning: Optuna ({N_OPTUNA_TRIALS} trials/step, TSS CV+SMOTE, optimizing Precision@{OPTUNA_EVAL_THRESHOLD})") # Updated print
    print(f"Total Individual Predictions Evaluated: {len(actual_np)}")
    print(f"Positive Target Occurrence (final eval set): {actual_np.mean()*100:.2f}%")
    print(f"Overall Accuracy:  {final_accuracy:.4f}")
    print(f"Overall Precision: {final_precision:.4f}")
    print(f"Overall Recall:    {final_recall:.4f}")
    print(f"Overall F1 Score:  {final_f1:.4f}")

    # Report metrics specifically at the threshold Optuna was evaluating
    if OPTUNA_EVAL_THRESHOLD in results_per_threshold:
        res_eval = results_per_threshold[OPTUNA_EVAL_THRESHOLD]
        print(f"(Metrics @ Optuna Eval Thresh {OPTUNA_EVAL_THRESHOLD}: F1:{res_eval['f1']:.4f}, Acc:{res_eval['acc']:.4f}, Pre:{res_eval['pre']:.4f}, Rec:{res_eval['rec']:.4f})") # <-- ADDED/MODIFIED
    elif 0.5 in results_per_threshold: # Fallback to show 0.5 if eval thresh wasn't hit
        res_def = results_per_threshold[0.5]
        print(f"(Compare: Default 0.5 Thresh -> F1:{res_def['f1']:.4f}, Acc:{res_def['acc']:.4f}, Pre:{res_def['pre']:.4f}, Rec:{res_def['rec']:.4f})")

    results_summary = { # Store results
        'symbol': SYMBOL_NAME, 'probabilities': probabilities_np, 'actuals': actual_np, 'timestamps': backtest_timestamps,
        'best_threshold_f1': best_threshold_f1, # Store the best F1 threshold
        'metrics_optimized_f1': {'acc': final_accuracy, 'pre': final_precision, 'rec': final_recall, 'f1': final_f1},
        'metrics_at_optuna_eval_thresh': results_per_threshold.get(OPTUNA_EVAL_THRESHOLD, {}), # Store metrics at Optuna's eval threshold
        'best_params_per_step': all_best_params, 'results_per_threshold': results_per_threshold
    }

    # --- 7. Plot Cumulative Accuracy ---
    print(f"\nPlotting cumulative accuracy for {SYMBOL_NAME} (using threshold optimized for F1)...")
    try:
        # Plotting uses final_predictions_optimized (based on best_threshold_f1)
        cumulative_accuracy_list_optimized = (np.cumsum(final_predictions_optimized == actual_np) / np.arange(1, len(actual_np) + 1))
        plt.figure(figsize=(14, 7))
        plt.plot(backtest_timestamps, cumulative_accuracy_list_optimized, marker='.', linestyle='-', markersize=1, alpha=0.7, label=f'Cumulative Accuracy ({SYMBOL_NAME}) @ F1 Threshold') # Clarify label
        rolling_window_plot_size = max(TEST_WINDOW_MINUTES * 5, 60 * 12)
        if len(actual_np) > rolling_window_plot_size:
             results_df = pd.DataFrame({'correct': (final_predictions_optimized == actual_np).astype(int)}, index=pd.to_datetime(backtest_timestamps))
             try: rolling_acc = results_df['correct'].rolling(window=rolling_window_plot_size, min_periods=rolling_window_plot_size//2).mean()
             except Exception as e_roll: print(f"Could not calculate rolling accuracy: {e_roll}"); rolling_acc = None
             if rolling_acc is not None: plt.plot(rolling_acc.index, rolling_acc, linestyle='--', color='red', label=f'Rolling Acc ({rolling_window_plot_size} min window)')
        plt.title(f'{SYMBOL_NAME} Backtest (Optuna Prec Opt, TSS CV+SMOTE) - Best F1 Thresh: {best_threshold_f1:.2f}') # Update title
        plt.xlabel('Timestamp'); plt.ylabel('Accuracy'); min_y_plot=max(0.0, np.min(cumulative_accuracy_list_optimized)-0.05 if len(cumulative_accuracy_list_optimized)>0 else 0.4)
        max_y_plot=min(1.0, np.max(cumulative_accuracy_list_optimized)+0.05 if len(cumulative_accuracy_list_optimized)>0 else 0.8)
        if max_y_plot - min_y_plot < 0.1: mid_point=(max_y_plot+min_y_plot)/2; min_y_plot=max(0.0, mid_point-0.05); max_y_plot=min(1.0, mid_point+0.05)
        plt.ylim(min_y_plot, max_y_plot); plt.grid(True, linestyle='--', alpha=0.6); plt.legend(); plt.xticks(rotation=30, ha='right'); plt.tight_layout()
        plot_filename = f"backtest_accuracy_{SYMBOL_NAME}_30m_target_Optuna_Prec_SMOTE.png" # Update filename
        plt.savefig(plot_filename); print(f"Saved accuracy plot to {plot_filename}"); plt.close()
    except Exception as e_plot: print(f"Error plotting: {e_plot}")

elif num_steps == 0: print(f"No backtesting steps completed.")
else: print(f"Error: Length mismatch in results arrays. Cannot evaluate.")

# --- End of Script ---
print(f"\n{'='*30} Overall Script Finished for {SYMBOL_NAME} {'='*30}")
overall_end_time = time.time()
print(f"Total execution time: {(overall_end_time - overall_start_time)/60:.2f} minutes.")

--- Data Loading ---
Loading data from: SOL_minagg.csv...
Loaded 22582 rows for SOL.

--- Feature Engineering for SOL ---
Feature engineering complete. Took 0.07s.

--- Target Definition ---
Defining target: 60m return >= 0.24%...
  Raw positive target occurrence (before NaN drop): 38.62%

--- Data Preparation ---
Applying final NaN/Inf Handling...
NaN Handling: Dropped 121 rows with NaNs.

Final feature matrix shape: (22461, 46), Target shape: (22461,)
Using 46 features.
Positive Target Rate in Final Data: 38.70%

--- Starting SLIDING Window Backtest for SOL ---
!!! Using Optuna (TimeSeriesSplit CV + SMOTE, optimizing Precision) + Rare Event Features !!!
Train Window: 480m, Step: 240m, Test Window: 96m, Optuna Trials: 50, CV Splits: 3

--- Step 1 (2025-04-03 10:01:00) ---
  Train: [0:479]; Test: [480:575]
  Training data balance before SMOTE: 19.38% positive
  Running Optuna (50 trials, cv=3 TimeSeriesSplit+SMOTE, scoring Precision@0.25)...
  Optuna finished in 31.72s.
  Best Params: 

Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x00000223087C1220>>
Traceback (most recent call last):
  File "C:\Users\mason\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py", line 534, in _next_wrapper
    return self._handle_exception(lambda: self.next(input_data), 0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mason\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py", line 469, in _handle_exception
    return fn()
           ^^^^
  File "C:\Users\mason\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py", line 534, in <lambda>
    return self._handle_exception(lambda: self.next(input_data), 0)
                                          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mason\AppData\Roaming\Python\Python312\site-packages\xgboost\data.py", line 1185, in next
    input_data(**self.kwargs)
  File "C:\Users\mas